In [11]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of source.custom_classes.data_loaders failed: Traceback (most recent call last):
  File "/home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance/faact_venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance/faact_venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 484, in superreload
    update_generic(old_obj, new_obj)
  File "/home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance/faact_venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 381, in update_generic
    update(a, b)
  File "/home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance/faact_venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 333, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/denys_heras

In [12]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [13]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


## Import dependencies

In [14]:
import os
import pandas as pd
from datetime import datetime, timezone

from configs.models_config_for_tuning import MODELS_CONFIG
from source.utils.custom_initializers import create_config_obj, create_models_config_from_tuned_params_df
from source.user_interfaces.metrics_computation_interfaces import compute_metrics_multiple_runs
from source.custom_classes.data_loaders import ACSEmploymentDataset

## Configs

In [15]:
config = create_config_obj(config_yaml_path=os.path.join('configs', 'experiment1_folktables_GA_2018_config.yaml'))

TUNED_PARAMS_FILE_PATH = os.path.join('results', 'models_tuning', f'tuning_results_{config.dataset_name}_20230117__023240.csv')
SAVE_RESULTS_DIR_PATH = os.path.join('results', 'hypothesis_space',
                                     f'{config.dataset_name}_Metrics_{datetime.now(timezone.utc).strftime("%Y%m%d__%H%M%S")}')

## Models tuned hyper-parameters

In [16]:
models_tuned_params_df = pd.read_csv(TUNED_PARAMS_FILE_PATH)
models_tuned_params_df

,Unnamed: 0,Dataset_Name,Model_Name,F1_Score,Accuracy_Score,Model_Best_Params
0,0,Folktables_GA_2018,LogisticRegression,0.8195,0.8203,"{'C': 1, 'max_iter': 150, 'penalty': 'l2', 'so..."
1,1,Folktables_GA_2018,DecisionTreeClassifier,0.8304,0.8313,"{'criterion': 'gini', 'max_depth': 10, 'max_fe..."
2,2,Folktables_GA_2018,RandomForestClassifier,0.8354,0.8357,"{'max_depth': 10, 'max_features': 0.6, 'min_sa..."
3,3,Folktables_GA_2018,XGBClassifier,0.8371,0.8375,"{'lambda': 1, 'learning_rate': 0.1, 'max_depth..."
4,4,Folktables_GA_2018,KNeighborsClassifier,0.8136,0.8137,"{'metric': 'minkowski', 'n_neighbors': 25, 'we..."
5,5,Folktables_GA_2018,MLPClassifier,0.8345,0.8353,"{'activation': 'logistic', 'hidden_layer_sizes..."


In [17]:
experiment_models_config = create_models_config_from_tuned_params_df(
    models_config_for_tuning=MODELS_CONFIG,
    models_tuned_params_df=models_tuned_params_df,
)

In [18]:
experiment_models_config.keys()

dict_keys(['DecisionTreeClassifier', 'LogisticRegression'])

## Load dataset

In [19]:
dataset = ACSEmploymentDataset(state=['GA'], year=2018, root_dir=os.path.join('data'), with_nulls=False, subsample=20000)
dataset.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
82065,5,0,7,1,2,1,2,2,2,2,2,2,1,1,2,6
90689,1,4,0,3,2,1,2,2,2,1,1,1,1,2,24,35
66829,3,4,0,1,2,1,2,2,2,2,2,0,1,1,20,38
94984,1,4,0,1,2,1,2,2,2,2,1,0,1,4,18,36
90278,5,4,0,3,2,1,2,2,2,1,1,0,1,4,16,43


## Run experiments

In [20]:
multiple_runs_metrics_dct = compute_metrics_multiple_runs(dataset, config, experiment_models_config, SAVE_RESULTS_DIR_PATH, debug_mode=False)

Analyze models in one run:   0%|          | 0/2 [00:00<?, ?it/s]

##############################  [Model 1 / 2] Analyze DecisionTreeClassifier  ##############################
Model random_state:  101
Baseline X_train shape:  (16000, 16)
Baseline X_test shape:  (4000, 16)




2023-01-31 22:49:01 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap: 100%|██████████| 100/100 [00:03<00:00, 31.99it/s]


2023-01-31 22:49:04 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-01-31 22:49:13 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics

Analyze models in one run:  50%|█████     | 1/2 [00:33<00:33, 33.67s/it]





##############################  [Model 2 / 2] Analyze LogisticRegression  ##############################
Model random_state:  102
Baseline X_train shape:  (16000, 16)
Baseline X_test shape:  (4000, 16)




...

2023-01-31 22:50:56 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap: 100%|██████████| 100/100 [00:03<00:00, 32.61it/s]


2023-01-31 22:50:59 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-01-31 22:51:10 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics

Analyze models in one run:  50%|█████     | 1/2 [00:38<00:38, 38.81s/it]





##############################  [Model 2 / 2] Analyze LogisticRegression  ##############################
Model random_state:  202
Baseline X_train shape:  (16000, 16)
Baseline X_test shape:  (4000, 16)




2023-01-31 22:51:35 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

Classifiers testing by bootstrap:  24%|██▍       | 24/100 [00:10<00:31,  2.38it/s]

Classifiers testing by bootstrap:  48%|████▊     | 48/100 [00:24<00:27,  1.90it/s]

Classifiers testing by bootstrap:  48%|████▊     | 48/100 [00:35<00:27,  1.90it/s]

Classifiers testing by bootstrap:  64%|██████▍   | 64/100 [00:35<00:21,  1.70it/s]

Classifiers testing by bootstrap:  79%|███████▉  | 79/100 [00:46<00:13,  1.56it/s]

Classifiers testing by bootstrap: 100%|██████████| 100/100 [01:00<00:00,  1.66it/s][A


2023-01-31 22:52:35 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-01-31 22:52:46 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics

Multiple runs progress: 100%|██████████| 2/2 [04:16<00:00, 128.25s/it]